In [1]:
%%bash
# Install required packages
apt-get update
apt install -y chromium-chromedriver
pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,809 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,266 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 kB]
Hit:13 https://ppa.launchpad

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Set up Chrome options for Colab
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Initialize the driver
driver = webdriver.Chrome(options=chrome_options)

symbol = 'hli'  # Change to your desired company symbol
url = f"https://www.sharesansar.com/company/{symbol}"

try:
    driver.get(url)
    print("Page loaded successfully")

    # Click the Price History tab
    price_history_tab = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Price History')]")))
    driver.execute_script("arguments[0].click();", price_history_tab)
    time.sleep(3)  # Wait for data to load

    all_data = []
    page_count = 1

    while True:
        print(f"Processing page {page_count}...")

        # Wait for table to load and refresh elements
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'dataTable')]/tbody/tr")))

        # Get fresh table reference every iteration
        table = driver.find_element(By.XPATH, "//table[contains(@class, 'dataTable')]")
        rows = table.find_elements(By.TAG_NAME, 'tr')

        for row in rows:
            try:
                # Re-find columns in case of staleness
                cols = row.find_elements(By.TAG_NAME, 'td')
                if len(cols) >= 8:  # Ensure all columns exist
                    row_data = {
                        'S.N': cols[0].text,
                        'date': cols[1].text,
                        'open_price': cols[2].text,
                        'high_price': cols[3].text,
                        'low_price': cols[4].text,
                        'close_price': cols[5].text,
                        'per_change': cols[6].text,
                        'volume': cols[7].text,
                        'turnover': cols[8].text
                    }
                    all_data.append(row_data)
            except Exception as e:
                print(f"Skipping row due to error: {e}")
                continue

        # Try to click next page
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]")))

            if 'disabled' in next_button.get_attribute('class'):
                print("Reached last page")
                break

            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(3)  # Wait for page load
            page_count += 1
        except Exception as e:
            print(f"Pagination error: {e}")
            break

finally:
    driver.quit()

# Save to CSV
if all_data:
    df = pd.DataFrame(all_data)
    from google.colab import files

    filename = f"{symbol}_price_history.csv"
    df.to_csv(filename, index=False)
    print(f"Successfully saved {len(df)} records")
    print("Sample data:")
    print(df.head())

    # Download the file
    files.download(filename)
else:
    print("No data was fetched")

Page loaded successfully
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Processing page 28...
Processing page 29...
Processing page 30...
Processing page 31...
Processing page 32...
Processing page 33...
Processing page 34...
Processing page 35...
Processing page 36...
Processing page 37...
Processing page 38...
Processing page 39...
Processing page 40...
Processing page 41...
Processing page 42...
Processing page 43...
Reached last page
Successfully saved 8

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>